In [1]:
%load_ext autoreload
%autoreload 2

import sys, os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from meta_neural_network_architectures import VGGReLUNormNetwork, ResNet12
from few_shot_learning_system import MAMLFewShotClassifier
from prompters import padding
from utils.parser_utils import get_args
from data import MetaLearningSystemDataLoader
from data import FewShotLearningDatasetParallel
from experiment_builder import ExperimentBuilder
import prompters

import easydict

import torch
import torch.nn as nn
import numpy as np

import torch.backends.cudnn as cudnn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR100
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim

In [2]:
os.environ['DATASET_DIR'] = os.path.join(os.getcwd(), "datasets")

args = easydict.EasyDict(
{
  "batch_size":2,
  "image_height":84,
  "image_width":84,
  "image_channels":3,
  "gpu_to_use":0,
  "num_dataprovider_workers":4,
  "max_models_to_save":5,
  "dataset_name":"mini_imagenet_full_size",
  "dataset_path":"mini_imagenet_full_size",
  "reset_stored_paths":False,
  "experiment_name":"alfa+maml",
  "train_seed": 0, "val_seed": 0,
  "indexes_of_folders_indicating_class": [-3, -2],
  "sets_are_pre_split": True,
  "train_val_test_split": [0.64, 0.16, 0.20],
  "evaluate_on_test_set_only": False,

  "total_epochs": 100,
  "total_iter_per_epoch":500, "continue_from_epoch": -2,
  "num_evaluation_tasks":600,
  "multi_step_loss_num_epochs": 15,
  "minimum_per_task_contribution": 0.01,
  "learnable_per_layer_per_step_inner_loop_learning_rate": False,
  "enable_inner_loop_optimizable_bn_params": False,
  "evalute_on_test_set_only": False,

  "max_pooling": True,
  "per_step_bn_statistics": False,
  "learnable_batch_norm_momentum": False,
  "load_into_memory": False,
  "init_inner_loop_learning_rate": 0.01,
  "init_inner_loop_weight_decay": 0.0005,
  "learnable_bn_gamma": True,
  "learnable_bn_beta": True,

  "dropout_rate_value":0.0,
  "min_learning_rate":0.001,
  "meta_learning_rate":0.001,   "total_epochs_before_pause": 100,
  "first_order_to_second_order_epoch":-1,
  "weight_decay": 0.0,

  "norm_layer":"batch_norm",
  "cnn_num_filters":48,
  "num_stages":4,
  "conv_padding": True,
  "number_of_training_steps_per_iter":5,
  "number_of_evaluation_steps_per_iter":5,
  "cnn_blocks_per_stage":1,
  "num_classes_per_set":5,
  "num_samples_per_class":5,
  "num_target_samples": 15,
    "samples_per_iter" : 1,

  "second_order": True,
  "use_multi_step_loss_optimization":False,
  "attenuate": False,
  "alfa": True,
  "random_init": False,
  "backbone": "4-CONV"
}
)

device = torch.cuda.current_device()
args.im_shape = (2, 3, args.image_height, args.image_width)

args.use_cuda = torch.cuda.is_available()
args.seed = 104
args.reverse_channels=False
args.labels_as_int=False
args.reset_stored_filepaths=False
args.num_of_gpus=1

In [3]:
preprocess = transforms.Compose([
    transforms.Resize(84),
    transforms.ToTensor()
])

train_dataset = CIFAR100("./data", transform=preprocess,
                          download=True, train=True)

val_dataset = CIFAR100("./data", transform=preprocess,
                        download=True, train=False)

train_loader = DataLoader(train_dataset,
                          batch_size=25, pin_memory=True,
                          num_workers=16, shuffle=True)

class_names = train_dataset.classes

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting ./data\cifar-100-python.tar.gz to ./data
Files already downloaded and verified


C:\Users\JM\anaconda3\envs\metal\lib\site-packages\torch\utils\data\dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [4]:
images, targets = next(iter(train_loader))
images = images.to(device)

targets = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4])
targets = torch.Tensor(targets)
targets = targets.type(torch.LongTensor)
targets = targets.to(device)

In [5]:
print(images.shape)
print(targets)

torch.Size([25, 3, 84, 84])
tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4,
        4], device='cuda:0')


In [35]:
class MAMLFewShotClassifier(nn.Module):
    
    def __init__(self, im_shape, device, args):
        
        super(MAMLFewShotClassifier, self).__init__()
        self.args = args
        self.device = device
        self.batch_size = args.batch_size
        self.use_cuda = args.use_cuda
        self.im_shape = im_shape
        self.current_epoch = 0
        
        self.classifier = VGGReLUNormNetwork(im_shape=self.im_shape, num_output_classes=self.args.
                                                 num_classes_per_set,
                                                 args=args, device=device, meta_classifier=True).to(device=self.device)
        
        
        self.prompter = prompters.padding(args=args, prompt_size=10, image_size=self.im_shape)
        
        
        self.optimizer_all = optim.Adam(self.trainable_parameters(), lr=args.meta_learning_rate, amsgrad=False)
        self.scheduler_all = optim.lr_scheduler.CosineAnnealingLR(optimizer=self.optimizer_all, T_max=self.args.total_epochs,
                                                              eta_min=self.args.min_learning_rate)
        
    def trainable_parameters(self):
        for param in self.parameters():
            if param.requires_grad:
                yield param
                
                
    def get_inner_loop_parameter_dict(self, params):
        
        param_dict = dict()
        for name, param in params:
            if param.requires_grad:
                param_dict[name] = param.to(device=device)

        return param_dict
    
    def forward(self, x, y):
        
        loss, prompt_preds = self.net_forward(x,y)
        observer = self.meta_update(loss)
        return observer
    
    
    def forward_grad(self, x, y):
        
        loss, prompt_preds = self.net_forward(x,y)
        
        prompt_grads = self.meta_update_grad(loss)
        
        return prompt_grads
        
    def forward_prompt_backward(self, x, y):
        
        loss, prompt_preds = self.net_forward(x,y)
        
        prompt_grads = self.meta_prompt_update(loss)
        
        return prompt_grads
    
    def forward_classifier_backward(self, x, y):
        
        loss, prompt_preds = self.net_forward(x,y)
        
        prompt_grads = self.meta_classifier_update(loss)
        
        return prompt_grads
    
        
    def net_forward(self, x, y):
        
        names_prompt_weights_copy = self.get_inner_loop_parameter_dict(self.prompter.named_parameters())
        names_weights_copy = self.get_inner_loop_parameter_dict(self.classifier.named_parameters())

        names_prompt_weights_copy = {
                        name.replace('module.', ''): value.unsqueeze(0).repeat(
                            [1] + [1 for i in range(len(value.shape))]) for
                        name, value in names_prompt_weights_copy.items()}

        names_weights_copy = {
                        name.replace('module.', ''): value.unsqueeze(0).repeat(
                            [1] + [1 for i in range(len(value.shape))]) for
                        name, value in names_weights_copy.items()}
        
        
        for param in self.prompter.parameters():
            param.requires_grad = False
        
        prompted_images = self.prompter.forward(x=images, params=names_prompt_weights_copy)
        prompt_preds = self.classifier.forward(prompted_images, params=names_weights_copy,num_step=4)
        
        loss = F.cross_entropy(input=prompt_preds, target=y)
        
        return loss, prompt_preds
        
    
    def meta_update_grad(self, loss):
        
        for param in self.classifier.parameters():
            print(param.requires_grad)
        
        prompt_grads = torch.autograd.grad(loss, self.classifier.parameters(), allow_unused=True)
        
        return prompt_grads
        
        
    def meta_prompt_update(self, loss):
        
        print("meta_prompt_update")
        
        for param in self.classifier.parameters():
            param.requires_grad = False
            
        for param in self.prompter.parameters():
            param.requires_grad = True
        
        self.optimizer_all.zero_grad()
        loss.backward()
        
        for name, param in self.prompter.named_parameters():
            print(param.grad)
        
        self.optimizer_all.step()
        
        
    def meta_classifier_update(self, loss):
        
        print("meta_classifier_update")
        
        for param in self.classifier.parameters():
            param.requires_grad = True
        
        for param in self.prompter.parameters():
            param.requires_grad = False
        
        self.optimizer_all.zero_grad()
        loss.backward()
        
        for name, param in self.classifier.named_parameters():
            print(param.grad)
        
        self.optimizer_all.step()
    
    
    def unify_test(self, images, targets):
        prompt_grads1 = model.forward_grad(images, targets)
        
        
        ## loss.bac
        model.forward_prompt_backward(images, targets)
        model.forward_classifier_backward(images, targets)
        
        return  prompt_grads1
    
    
    def meta_update(self, loss):
        
        print("loss == ", loss)
        
        #print(self.classifier.state_dict())
        print(self.prompter.state_dict())
        print("==============================")
        
        #### Prompter Update ####
        
        for param in self.classifier.parameters():
            param.requires_grad = False

        for param in self.prompter.parameters():
            param.requires_grad = True
        
        self.optimizer_all.zero_grad()
        
        loss.backward(retain_graph=True)
        self.optimizer_all.step()

        print(self.prompter.state_dict())
        print("==============================")
        
        
        #### Classifier Update ####        
                
        self.classifier.zero_grad()
        self.prompter.zero_grad()
        self.optimizer_all.zero_grad()
        
        for param in self.classifier.parameters():
            param.requires_grad = True

        for param in self.prompter.parameters():
            param.requires_grad = False
            
        loss.backward()
        self.optimizer_all.step()


        print(self.prompter.state_dict())
        print("==============================")
        
        
#########
model = MAMLFewShotClassifier(args=args, device=device, im_shape=(2, 3, args.image_height, args.image_width))

Using max pooling
No inner loop params
torch.Size([2, 48, 84, 84])
No inner loop params
No inner loop params
torch.Size([2, 48, 42, 42])
No inner loop params
No inner loop params
torch.Size([2, 48, 21, 21])
No inner loop params
No inner loop params
torch.Size([2, 48, 10, 10])
No inner loop params
(VGGReLUNormNetwork) meta network params
layer_dict.conv0.conv.weight torch.Size([48, 3, 3, 3])
layer_dict.conv0.conv.bias torch.Size([48])
layer_dict.conv0.norm_layer.running_mean torch.Size([48])
layer_dict.conv0.norm_layer.running_var torch.Size([48])
layer_dict.conv0.norm_layer.bias torch.Size([48])
layer_dict.conv0.norm_layer.weight torch.Size([48])
layer_dict.conv1.conv.weight torch.Size([48, 48, 3, 3])
layer_dict.conv1.conv.bias torch.Size([48])
layer_dict.conv1.norm_layer.running_mean torch.Size([48])
layer_dict.conv1.norm_layer.running_var torch.Size([48])
layer_dict.conv1.norm_layer.bias torch.Size([48])
layer_dict.conv1.norm_layer.weight torch.Size([48])
layer_dict.conv2.conv.weight

In [36]:
grad = model.forward_grad(images, targets)

print(grad)

True
True
False
False
True
True
True
True
False
False
True
True
True
True
False
False
True
True
True
True
False
False
True
True
True
True


RuntimeError: One of the differentiated Tensors does not require grad

### 1) retain_graph= True 사용법 확인

In [7]:
observer = model.forward(images, targets)

loss ==  tensor(2.1328, device='cuda:0', grad_fn=<NllLossBackward>)
OrderedDict([('pad_dict.pad_up', tensor([[[-0.4146,  0.1824,  1.5668,  ...,  0.1607,  2.1045,  1.0947],
         [-0.5065,  0.7279, -0.3725,  ...,  1.4798,  0.7338,  0.6663],
         [-1.2741,  0.3628,  0.7863,  ..., -0.0027, -1.0407,  0.1453],
         ...,
         [ 0.2212, -0.4895, -0.3429,  ...,  0.0668, -0.1131, -1.5486],
         [ 0.7786,  0.7920, -1.4037,  ...,  0.0371, -0.6172,  2.3837],
         [ 0.7835, -0.7619,  0.7989,  ...,  1.6326,  0.7327,  0.7296]],

        [[-0.7939, -0.3529,  0.5046,  ...,  0.1915,  0.1670, -0.1800],
         [-0.3602,  0.2038, -1.6841,  ...,  0.9995,  1.6749, -1.1009],
         [ 0.7550, -0.4750,  1.3468,  ...,  1.2652,  1.3309,  0.3080],
         ...,
         [ 0.6359, -0.0728,  0.0797,  ...,  0.3535,  0.6812,  0.1956],
         [-0.1643, -0.1860, -0.4835,  ...,  1.2388, -2.0533,  1.8488],
         [-0.7935,  0.2394, -0.0627,  ..., -0.7649, -0.4766, -0.7666]],

        [[-1.71

In [8]:
before_update_classifier = observer['before_update_classifier']
before_update_prompter = observer['before_update_prompter']

one_update_classifier = observer['one_update_classifier']
one_update_prompter = observer['one_update_prompter']

two_update_classifier = observer['two_update_classifier']
two_update_prompter = observer['two_update_prompter']


for param in before_update_classifier.parameters():
    print(param)

TypeError: 'NoneType' object is not subscriptable

In [ ]:
for param in one_update_classifier.parameters():
    print(param)

In [ ]:
for param in two_update_classifier.parameters():
    print(param)

In [ ]:
for param in before_update_prompter.parameters():
    print(param)

In [ ]:
for param in one_update_prompter.parameters():
    print(param)

In [ ]:
for param in two_update_prompter.parameters():
    print(param)

### 2) torch.autograd.grad와 loss.backward를 사용해서 gradient를 구한 것이 같은지 확인

In [ ]:
prompt_grads1 = model.unify_test(images, targets)